<a href="https://colab.research.google.com/github/HiroshiATakano/QC_Kentei/blob/main/Level1_12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Diseño de experimentos

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
def ensure_and_move_to_folder(folder_name):
  current_dir = os.getcwd()
  target_path = os.path.join(current_dir, folder_name)

  if not os.path.exists(target_path):
    os.makedirs(target_path)
    print(f"Folder '{folder_name}' created successfully.")
  else:
    print(f"Folder '{folder_name}' already exists.")

  #os.chdir(target_path)

In [4]:
import os

folder_name = 'QC_Kentei'
ensure_and_move_to_folder(folder_name)

path = os.getcwd()

try:
  os.chdir(path + '/drive/My Drive/QC_Kentei')
except:
  pass

os.getcwd()

Folder 'QC_Kentei' created successfully.


'/content/drive/My Drive/QC_Kentei'

In [ ]:
!pip install japanize-matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 10.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for japanize-matplotlib: filename=japanize_matplotlib-1.1.3-py3-none-any.whl size=4120257 sha256=22095d88fe06212b223ab715bf798fccdbb0bcee420b76424ce61493ec809501
  Stored in directory: /root/.cache/pip/wheels/da/a1/71/b8faeb93276fed10edffcca20746f1ef6f8d9e071eee8425fc
Successfully built japanize-matplotlib


In [ ]:
folder_name = 'Data_1'
ensure_and_move_to_folder(folder_name)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import japanize_matplotlib
import gspread

from google.colab import auth
auth.authenticate_user()

from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

from gspread_dataframe import get_as_dataframe, set_with_dataframe

## 12.1

L8直交配列表を用いた実験の結果、表12.1のデータを得た。

1. 直交配列表を用いた分散分析表を作成せよ

Resultado del experimento utilizando la tabla de diseño ortogonal L18.

1. Elabore una tabla de análisis de varianza utilizando una matriz ortogonal.

In [ ]:
url = "https://docs.google.com/spreadsheets/d/1Ma7ehT4TAnkUwJjspP-C3mwJA1hYmbJr8B9GJSFI0lU/edit?usp=sharing"
ss = gc.open_by_url(url)

# Obtener los datos (como lista)
st = ss.worksheet("Sheet5")
list_of_lists = st.get_all_values()

df = pd.DataFrame.from_records(list_of_lists)
df.columns = df.iloc[0]
df = df.drop(index=0)

df = df.apply(pd.to_numeric)

df.to_excel("Data_1/12_1.xlsx",index=False)

data = df.to_dict(orient="list")

In [ ]:
import polars as pl

# Crear un DataFrame con Polars
df2 = pl.DataFrame(data)

# Mostrar los datos
print("Tabla 12.1 Tabla de diseño ortogonal L8 y resultados del experimento")
print(df2)

In [ ]:
def calc_ss(df: pl.DataFrame, factor: str, data_col: str, total_mean: float) -> float:
    """
    Función para calcular la suma de los cuadrados en el factor especificado.
    """
    # Estadísticas de un grupo
    df_grp1 = df.filter(pl.col(factor) == 1)
    mean_1 = df_grp1.select(pl.col(data_col).mean()).item()
    n1 = df_grp1.height

    # Estadísticas de dos grupos
    df_grp2 = df.filter(pl.col(factor) == 2)
    mean_2 = df_grp2.select(pl.col(data_col).mean()).item()
    n2 = df_grp2.height

    # Cálculo de la suma de los cuadrados
    ss = n1 * ((mean_1 - total_mean) ** 2) + n2 * ((mean_2 - total_mean) ** 2)
    return ss

# Suma total de los cuadrados (Total Sum of Squares)
total_mean = df['Data'].mean()
total_ss = sum((df['Data'] - total_mean) ** 2)

# Calcular la suma de los cuadrados de cada factor
ss_A = calc_ss(df2, "Factor_A", "Data", total_mean)
ss_B = calc_ss(df2, "Factor_B", "Data", total_mean)
ss_C = calc_ss(df2, "Factor_C", "Data", total_mean)
ss_D = calc_ss(df2, "Factor_D", "Data", total_mean)

# Suma de los cuadrados de los residuos
ss_AxB = calc_ss(df2, "AxB", "Data", total_mean)
ss_AxC = calc_ss(df2, "AxC", "Data", total_mean)

# Suma de los cuadrados de los residuos (Residual Sum of Squares)
ss_residual = total_ss - (ss_A + ss_B + ss_C + ss_D + ss_AxB + ss_AxC)

print(f"Suma total de los cuadrados: {total_ss}")

print("Suma de los cuadrados del fator A:", ss_A)
print("Suma de los cuadrados del fator B:", ss_B)
print("Suma de los cuadrados del fator C:", ss_C)
print("Suma de los cuadrados del fator D:", ss_D)

print("Suma de los cuadrados de AxB:", ss_AxB)
print("Suma de los duadrados de AxC:", ss_AxC)

print("Suma de los cuadrados del erro (Residual SS):", ss_residual)


### Creación de la tabla de análisis de varianza (tabla ANOVA)

In [ ]:
# Grados de libertad (df) de cada término
df_A   = 2 - 1
df_B   = 2 - 1
df_C   = 2 - 1
df_D   = 2 - 1
df_AxB = 2 - 1
df_AxC = 2 - 1

# Los grados de libertad de los residuos se calculan restando la suma de
# los niveles de cada efecto del total de datos
n_total = df2.height
df_total = n_total - 1
df_model = df_A + df_B + df_C + df_D + df_AxB + df_AxC
df_residual = df_total - df_model

# Cálculo del cuadrado medio (Mean Square: MS)
ms_A   = ss_A   / df_A
ms_B   = ss_B   / df_B
ms_C   = ss_C   / df_C
ms_D   = ss_D   / df_D
ms_AxB = ss_AxB / df_AxB
ms_AxC = ss_AxC / df_AxC
ms_residual = ss_residual / df_residual

# Cálculo del valor F (dividiendo el MS de cada efecto por el MS de los residuos)
F_A   = ms_A   / ms_residual
F_B   = ms_B   / ms_residual
F_C   = ms_C   / ms_residual
F_D   = ms_D   / ms_residual
F_AxB = ms_AxB / ms_residual
F_AxC = ms_AxC / ms_residual

# Tabla ANOVA
anova_table = pl.DataFrame({
    "Source": ["Factor_A", "Factor_B", "Factor_C", "Factor_D", "AxB", "AxC", "Residual", "Total"],
    "SS":      [ss_A, ss_B, ss_C, ss_D, ss_AxB, ss_AxC, ss_residual, total_ss],
    "df":      [df_A, df_B, df_C, df_D, df_AxB, df_AxC, df_residual, df_total],
    "MS":      [ms_A, ms_B, ms_C, ms_D, ms_AxB, ms_AxC, ms_residual, None],
    "F":       [F_A, F_B, F_C, F_D, F_AxB, F_AxC, None, None]
})

print(anova_table)

2. 分散分析の結果、主効果Cおよび交互作用AxCは、有意ではなく、分散比も小さいので、これらを誤差にプールした分散分析表を作成せよ。


2. Como resultado del análisis de varianza, los efectos principales C y la interacción AxC no son significativos, y como la relación de varianza es pequeña, se debe crear una tabla de análisis de varianza donde se agrupan en el error.

In [ ]:
# --- Tratamiento de agrupamiento ---
# Agrupa los efectos del factor C y AxC en el error
ss_residual_pool = ss_residual + ss_C + ss_AxC
df_residual_pool = df_residual + df_C + df_AxC

# --- Cálculo del cuadrado medio (MS) de los residuos ---
ms_residual_pool = ss_residual_pool / df_residual_pool

# --- Cálculo del valor F ---
F_A   = np.round(ms_A   / ms_residual_pool,2)
F_B   = np.round(ms_B   / ms_residual_pool,2)
F_D   = np.round(ms_D   / ms_residual_pool,2)
F_AxB = np.round(ms_AxB / ms_residual_pool,2)

# --- Creación de la tabla de análisis de varianza (ANOVA table)  ---
anova_table = pl.DataFrame({
    "Source": ["Factor_A", "Factor_B", "Factor_D", "AxB", "Residual (Pooled)", "Total"],
    "SS":      [ss_A, ss_B, ss_D, ss_AxB, ss_residual_pool, total_ss],
    "df":      [df_A, df_B, df_D, df_AxB, df_residual_pool, df_total],
    "MS":      [ms_A, ms_B, ms_D, ms_AxB, ms_residual_pool, None],
    "F":       [F_A, F_B, F_D, F_AxB, None, None]
})

print(anova_table)

3. 分散分析の結果、データの構造式は：


3. Como resultado del análisis de varianza, la fómula estructural de los deatos es:

* $ x = \mu + a + b + d + (ab) + \epsilon $




4. データの総和を $ T $、因子 $ X $ の $ i $ 水準でのデータの和を $ T(X_i) $、因子 $ X $, $ Y $ の $ i $ 水準、 $ j $ 水準の組み合わせでのデータの和を $ T(X_iY_j) $ と表記した時、因子 $ A,B,D $ の水準組み合わせの母平均 $ \hat{\mu}(A_i B_j D_k) $ を推定する。


4. Cuando la suma total de los datos se denota por  $ T $, la suma de los datos en el nivel $ i $ del factor  $ X $ se denota por $ T(X_i) $, y la suma de los datos en la combinación de los niveles  $ i $ del factor $ X $ y  $ j $ del factor  $ Y $ se denota por $ T(X_iY_j) $, se estima la media poblacional $ \hat{\mu}(A_i B_j D_k) $ para la combinación de niveles de los factores $ A,B,D $.


* $ \hat{\mu}(A_i B_j D_k)  = \frac{T(A_iB_j)}{2} + \frac{T(D_k)}{4} - \frac{T}{8} $

In [ ]:
# Calculra la suma de los elementos
T_AiBj = ss_A + ss_AxB  # Suma total de los cuadrados de AiBj
T_Dk = ss_D  # Suma de los cuadrados de Dk

# Cálculo la media poblacional
mu_AiBjDk = (T_AiBj / 2) + (T_Dk / 4) - (total_ss / 8)

# Mostrar el resultado
print(f"Media poblacional μ(AiBjDk) = {mu_AiBjDk}")

5. 回帰モデル型分散分析により分散分析表を作成する


5. Crear una tabla de análisis de varianza mediante análisis de varianza de modelo de regresión

In [ ]:
df3 = df2.drop(['AxB','AxC','Residual'])
print(df3)

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

df = df3.to_pandas()

def convertir_a_categoricas(data, columnas):
    for columna in columnas:
        data[columna] = data[columna].astype('category')
    return data

columnas_a_convertir = ['Factor_A', 'Factor_B', 'Factor_C', 'Factor_D']
df = convertir_a_categoricas(df, columnas_a_convertir)

# Definir el modelo de análisis de varianza incluyendo las interacciones AxB y AxC
model = smf.ols('Data ~ Factor_A * Factor_B + Factor_A * Factor_C + Factor_D', data=df).fit()

# Crear la tabla de análisis de varianza
anova_table = sm.stats.anova_lm(model, typ=2)
print(anova_table)


In [ ]:
# Agrupar los factores no significativos
# (integrar en el error aquellos con una razón de varianza pequeña)
insignificant_factors = anova_table[anova_table['PR(>F)'] > 0.25].index.tolist()
formula_reduced = 'Data ~ ' + ' + '.join([factor for factor in ['Factor_A', 'Factor_B', 'Factor_C', 'Factor_D', 'Factor_A:Factor_B', 'Factor_A:Factor_C'] if factor not in insignificant_factors])

# Realizar nuevamente el análisis de varianza con un modelo simplificado
model_reduced = smf.ols(formula_reduced, data=df).fit()
anova_table_reduced = sm.stats.anova_lm(model_reduced, typ=2)
print("Reduced ANOVA Table:")
print(anova_table_reduced)


## 12.3

因子A（３水準）因子B（２水準）を表12.3のL8直交配列表に割り付け、No1～No8の実験をランダムな順序で行った。表12.3の実験結果を得た。

なお、列 a, bの水準と因子Aの水準設定は、表12.4のように行った。

下記を求めよ。
1. 因子Aの平方和と自由度
2. 因子Bの平方和と自由度
3. 交互作用AxBの平方和と自由度
4. 誤差の平方和と自由度


Se asignaron el factor A (con 3 niveles) y el factor B (con 2 niveles) a la matriz ortogonal L8 de la Tabla 12.3 y los experimentos No1 a No8 se realizaron en un orden aleatorio. Se obtuvieron los resultados experimentales de la Tabla 12.3.

Cabe destacar que los niveles de las columnas a y b y la configuración de los niveles del factor A se establecieron según la Tabla 12.4.

Determinar lo siguiente:
1. La suma de cuadrados y los grados de libertade del factor A.
2. La suma de cuadrados y los grados de libertad del factor B.
3. La sumade cuadrados y los grados de libertad de la interacción AxB.
4. La suma de cuadrados y los grados de libertad del residuo.

In [ ]:
# Obtener los datos (como lista)
st = ss.worksheet("Sheet8")
list_of_lists = st.get_all_values()

df = pd.DataFrame.from_records(list_of_lists)
df.columns = df.iloc[0]
df = df.drop(index=0)
df = df.apply(pd.to_numeric)

print("Tabla 12.3 Matriz ortogonal L8 y Resultados experimentales")
df

In [ ]:
# Configuración del Factor A y B

df = df.rename(columns={'c': 'Factor_B'})

def assign_factor_a(row):
    if row['a'] == 1 and row['b'] in [1, 2]:
        return 1
    elif row['a'] == 2 and row['b'] == 1:
        return 2
    elif row['a'] == 2 and row['b'] == 2:
        return 3

df['Factor_A'] = df.apply(assign_factor_a, axis=1)

print("Tabla 12.4 Configuración de los niveles del factor A")
df[['a','b','Factor_A','Factor_B']]

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

# Creación de datos
data_dict = df.to_dict(orient="list")

# Conversión a un DataFrame
data = pd.DataFrame(data_dict)

# Conversión de los factores A y B en variables cotegóricas
columnas_a_convertir = ['Factor_A', 'Factor_B']
data = convertir_a_categoricas(data, columnas_a_convertir)

# Definición del modelo de análisis de varianza
model = ols('x ~ Factor_A + Factor_B + Factor_A:Factor_B', data=data).fit()

# Tabla de análisis de varianza (ANOVA)
anova_table = sm.stats.anova_lm(model, typ=2)

# Visualización de resultados
print("Tabla de análisis de varianza (AVOVA):")
print(anova_table)

# Obtención de las suma de cuadrados
ssq_A = anova_table['sum_sq']['Factor_A']
ssq_B = anova_table['sum_sq']['Factor_B']
ssq_AB = anova_table['sum_sq']['Factor_A:Factor_B']
error = anova_table['sum_sq']['Residual']

# Obtención de los grados de libertad
df_A = anova_table['df']['Factor_A']
df_B = anova_table['df']['Factor_B']
df_AB = anova_table['df']['Factor_A:Factor_B']
df_error = anova_table['df']['Residual']


print(f"\nSuma de cuadrados del factor A: {ssq_A:.0f}  Grados de cuadrados del factor A： {df_A}")
print(f"Suma de cuadrados del factor A: {ssq_B:.0f}     Grados de cuadrados del factor B:  {df_B}")
print(f"Suma de cuadrados de la interacción AxB: {ssq_AB:.0f}   Grados de libertad de la interacción AxB:  {df_AB}")
print( f"Suma de cuadrados del residuo: {error:.0f}   Grados de libertad del residuo:  {df_error}")


## 12.6

因子A（３水準）を一次因子、因子B（２水準）、因子C（３水準）とした反復（R）２回の２段分割実験を行い、表１２．７のデータを得た。
1. 交互作用AxCの平方和 $ S_{A \times C} $
2. 一次誤差の平方和、自由度
3. ２次誤差の平方和、自由度
4. ３次誤差の平方和、自由度

を求めよ。

Se realizó un experimento de división en dos etapas con dos repeticiones (R), donde el factor A (3 niveles) se consideró como el factor primero, y los factor  B (2 niveles) y C (3 niveles) se incluyeron en la estructura experimental. Como resultado, se obtuvieron los datos de la Tabla 12.7.

Calcule los siguientes valores:
1. La suma de cuadrados de la interacción $ S_{A \times C} $
2. La suma de cuadrados y los grados de libertad del erro de primer nivel.
3. La suma de cuadrados y los grados de libertad del error de segundo nivel.
4. La suma de cuadrados y los grados de libertad de error de tercer nivel.

In [ ]:
# Obtener los datos (como lista)
st = ss.worksheet("Sheet9")
list_of_lists = st.get_all_values()

df = pd.DataFrame.from_records(list_of_lists)
df.columns = df.iloc[0]
df = df.drop(index=0)
df = df.apply(pd.to_numeric)

# Crear una tabla dinámica
df_pivot = df.pivot(index=['R', 'A', 'B'], columns='C', values='y')
# Modificar los nombres de las columnas según corresponda
df_pivot.columns = [f'C{col}' for col in df_pivot.columns]

print("Tabla 12.7 Resultados experimentales")
print(df_pivot.reset_index())

In [ ]:
# Restaurar la tabla dinámica a su form aoriginal
df_unpivot = df_pivot.reset_index().melt(id_vars=['R', 'A', 'B'], var_name='C', value_name='y')
df_unpivot['C'] = df_unpivot['C'].str.extract(r'C(\d+)').astype(int)

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

# Creación de datos
data_dict = df.to_dict(orient="list")

# Conversión a un DataFrame
data = pd.DataFrame(data_dict)

# Conversión de los factores A y B en variables cotegóricas
def convertir_a_categoricos(data, columns):
  for col in columns:
    data[col] = data[col].astype('category')
  return data

columnas_a_convertir = ['A', 'B', 'C', 'R']
data = convertir_a_categoricos(data, columnas_a_convertir)


# Definición del modelo de análisis de varianza
model = ols('y ~ R + A + R:A + B + A:B + R:B + C + A:C + B:C + R:C + A:B:C + R:A:C + R:B:C + R:A:B ', data=data).fit()

# Tabla de análisis de varianza (ANOVA)
anova_table = sm.stats.anova_lm(model, typ=2)

# Visualización de resultados
print("Tabla de análisis de varianza (AVOVA):")
print(anova_table)

反復R、因子A、B、Cの効果を $ r, a, b, c $、一次誤差、２次誤差、３次誤差を $ \epsilon (1), \epsilon(2), \epsilon(3) $ とした場合、実験をあらわすデータの構造式：

Si denotamos la repetición como $ R $, los factores como $ A, B, C $, y los errores como $ \epsilon (1), \epsilon(2), \epsilon(3) $, entonces la estructura del modelo experimental se expresa como:

$ x_{ijkl} = \mu + r_l + a_i + \epsilon(1)_{il} + b_j + (ab)_{ij} + \epsilon(2)_{ijl} + c_k + (ac)_{ik} + (bc)_{jk} + (abc)_{ijk} + \epsilon(3)_{ijkl} $



In [ ]:
# Error de primer nivel
ss1 = anova_table.loc['R:A', 'sum_sq']
df1 = anova_table.loc['R:A', 'df']

# Error de segundo nivel
ss2 = (anova_table.loc['R:A:B', 'sum_sq'] +
       anova_table.loc['R:B', 'sum_sq'])
df2 =   (anova_table.loc['R:A:B', 'df'] +
       anova_table.loc['R:B', 'df'])

# Error de tercer nivel
ss3 = (anova_table.loc['R:C', 'sum_sq'] +
      anova_table.loc['R:A:C', 'sum_sq'] +
      anova_table.loc['R:B:C', 'sum_sq'] +
      anova_table.loc['Residual', 'sum_sq'])
df3 = (anova_table.loc['R:C', 'df'] +
      anova_table.loc['R:A:C', 'df'] +
      anova_table.loc['R:B:C', 'df'] +
      anova_table.loc['Residual', 'df'])

print(f'Error de primer nivel (A×R): {ss1:.0f}', df1)
print(f'Error de segundo nivel (B×R+AxBxR): {ss2:.0f}', df2)
print(f'Error de tercer nivel (CxR+AxCxR+BxCxR+AxBxCxR): {ss3:.0f}', df3)


## 12.7

原料を一次処理して、一次中間体Pを製造し、Pを２次処理して２次中間体Qを製造し、Qを３次処理して製品を製造する工程がある。一次処理には因子Aが、２次処理には因子B、Cが、３次処理には因子D、Fが影響を及ぼすと考えられる。この度、これらの因子の製品の特性への影響をけんとうするため、以下の実験を行った。

因子Aを一次因子、因子B,Cを２次因子、因子D,Fを３次因子として、表１２．８のようにL18直交配列表に割り付けた。一次誤差を一次因子と交絡させないための実験を行う手順を決める。

Si tiene un proceso en el que la materia a un tratamiento primario para producir un intermedio primario P, luego P se somete a un tratamiento secundario para obtener un intermedio secundario Q, y finalmente Q se somete a un tratamiento terciario para fabricar el producto final. Se considera que el tratamiento primario está influenciado por el factor A, el tratamiento secundario por los factores B y C, y el tratamiento terciario por los factores D y F. Con el fin de evaluar la influencia de estos factores en las características del producto, se llevó a cabo el siguiente experimento.

Se asignaron el factor A como factor de primer nivel, los factores B y C como factores de segundo nivel, y los factores D y F como factores de tercer nivel en la tabla ortogonal L18, como se muestra en la Tabla 12.8. Se determinará el procedimiento experimental para evitar que el error de primer nivel se confunda con el factor de primer nivel.




## 実験の順番

1. 一次誤差を一次因子Aと交絡させないためには、一次因子Aの水準が均等に分布するように実験グループを選ぶ必要がある。
2. 因子B、Cの水準が均等に分配されるように設計する必要がある。
3. ３次因子D,Fの水準の均等性を考量する。


最終的な実験順番
1. 一次処理（Pの製造）：(1,2)(3,4)(5,6)(7,8)(9,10)(11,12)(13,14)(15,16)
2. 二次処理（Qの製造）：(3,4)(5,6)(7,8)(9,10)
3.３次処理（製品の製造）：(11,12)(13,14)(15,16)


## Orden de los experimentos

1. Para evitar que el error de primer orden se confunda con el factor A, es necesario elegir grupos experimentales de manera que los niveles del factor A se distribuyan uniformemente.
2. Es necesario diseñar el experimento de manera de los niveles de los factores B y C se distribuyan de manera uniforme.
3. Se debe considerar la igualidad de los niveles de los factores de tercer orden D y F.

Orden final de los experimentos:
1. Primer tratamiento (fabricación de P): (1,2)(3,4)(5,6)(7,8)(9,10)(11,12)(13,14)(15,16)
2. Segundo tratamiento (fabricación de P): (3,4)(5,6)(7,8)(9,10)
3. Tercer tratamiento (fabricación del producto): (11,12)(13,14)(15,16)



In [ ]:
import itertools
import random

# Obtener los datos (como lista)
st = ss.worksheet("Sheet14")
list_of_lists = st.get_all_values()

df = pd.DataFrame.from_records(list_of_lists)
df.columns = df.iloc[0]
df = df.drop(index=0)
df = df.apply(pd.to_numeric)

def assign_factors_and_execute_experiment(df):
    # Assign factors based on the given conditions
    df['A'] = df['1']  # Primary process factor
    df['BxC'] = df['3']
    df['B'] = df['4']  # Secondary process factor
    df['DxF'] = df['6']  # Secondary process factor
    df['C'] = df['7']  # Secondary process factor
    df['D'] = df['10']  # Tertiary process factor
    df['CxF'] = df['11']  # Tertiary process factor
    df['F'] = df['12']  # Tertiary process factor

    # Output allocation table
    allocation_table = df[['A','BxC', 'B', 'DxF', 'C', 'D', 'CxF','F']]
    print("Tabla12.8 Factor Allocation Table:")
    print(allocation_table)

    return allocation_table

# Assign factors and execute experiment
allocation_table = assign_factors_and_execute_experiment(df)
allocation_table = allocation_table.reset_index()

allocacion_list = allocation_table.to_dict(orient="list")

In [ ]:
# Condiciones del experimento
np.random.seed(5)  #
mu = 50  # Media total
sigma_A = np.sqrt(10)  # Desviación estándar del factor A
sigma_B = np.sqrt(3)  # Desviación estándar del factor B
sigma_C = np.sqrt(4)  # Desviación estándar del factor C
sigma_D = np.sqrt(5)  # Desviación estándar del factor D
sigma_F = np.sqrt(6)  # Desviación estándar del factor F
sigma_error = np.sqrt(1)  # Desviación estándar del error
sigma_DF = np.sqrt(5)  # Desviación estándar de la interacción entre D y F
sigma_BC = np.sqrt(1)  # Desviación estándar de la interacción entre B y C
sigma_CF = np.sqrt(2)  # Desviación estándar de la interacción entre C y F

# Efecto del nivel de los factores
A_levels = {1: np.random.normal(0, sigma_A), 2: np.random.normal(0, sigma_A)}
B_levels = {1: np.random.normal(0, sigma_B), 2: np.random.normal(0, sigma_B)}
C_levels = {1: np.random.normal(0, sigma_C), 2: np.random.normal(0, sigma_C)}
D_levels = {1: np.random.normal(0, sigma_D), 2: np.random.normal(0, sigma_D)}
F_levels = {1: np.random.normal(0, sigma_F), 2: np.random.normal(0, sigma_F)}

# Interacción
DF_interaction = {(d, f): np.random.normal(0, sigma_DF) for d in [1, 2] for f in [1, 2]}
BC_interaction = {(b, c): np.random.normal(0, sigma_BC) for b in [1, 2] for c in [1, 2]}
CF_interaction = {(c, f): np.random.normal(0, sigma_CF) for c in [1, 2] for f in [1, 2]}

# Tabla de arreglo experimental
df = pd.DataFrame(allocacion_list)

# Cálculo de x
x_values = [
    mu + A_levels[a] + B_levels[b] + C_levels[c] + D_levels[d] +F_levels[f] + BC_interaction[(b,c)] + DF_interaction[(d,f)] + CF_interaction[(c,f)] + np.random.normal(0, sigma_error)
    for a, b, c, d, f, in zip(df['A'], df['B'], df['C'], df['D'], df['F'])
]
df['x'] = np.round(x_values,1)


In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy import stats

# Creación de grupos para el análisis de varianza (ANOVA)
grouped_A = df.groupby('A')['x'].apply(list)
grouped_B = df.groupby('B')['x'].apply(list)
grouped_C = df.groupby('C')['x'].apply(list)
grouped_D = df.groupby('D')['x'].apply(list)
grouped_F = df.groupby('F')['x'].apply(list)
grouped_DxF = df.groupby('DxF')['x'].apply(list)
grouped_CxF = df.groupby('CxF')['x'].apply(list)

# Ejecución del ANOVA por cada factor
f_value_A, p_value_A = stats.f_oneway(*grouped_A)
f_value_B, p_value_B = stats.f_oneway(*grouped_B)
f_value_C, p_value_C = stats.f_oneway(*grouped_C)
f_value_D, p_value_D = stats.f_oneway(*grouped_D)
f_value_F, p_value_F = stats.f_oneway(*grouped_F)
f_value_DxF, p_value_DxF = stats.f_oneway(*grouped_DxF)
f_value_CxF, p_value_CxF = stats.f_oneway(*grouped_CxF)

# Cálculo de la suma de cuadrados y los grados de libertad de todos los factores
# Suma total de cudrados
df_A = len(grouped_A) - 1
df_B = len(grouped_B) - 1
df_C = len(grouped_C) - 1
df_D = len(grouped_D) - 1
df_F = len(grouped_F) - 1
df_DxF = len(grouped_DxF) - 1
df_CxF = len(grouped_CxF) - 1

# Suma total de cuadrados de los residuos（Total Sum of Squares）
total_ss = np.sum((df['x'] - np.mean(df['x']))**2)

# Suma de cuadrados por cada factor
ss_A = np.sum([len(group) * (np.mean(group) - np.mean(df['x']))**2 for group in grouped_A])
ss_B = np.sum([len(group) * (np.mean(group) - np.mean(df['x']))**2 for group in grouped_B])
ss_C = np.sum([len(group) * (np.mean(group) - np.mean(df['x']))**2 for group in grouped_C])
ss_D = np.sum([len(group) * (np.mean(group) - np.mean(df['x']))**2 for group in grouped_D])
ss_F = np.sum([len(group) * (np.mean(group) - np.mean(df['x']))**2 for group in grouped_F])
ss_DxF = np.sum([len(group) * (np.mean(group) - np.mean(df['x']))**2 for group in grouped_DxF])
ss_CxF = np.sum([len(group) * (np.mean(group) - np.mean(df['x']))**2 for group in grouped_CxF])

# Suma de cuadrados de los residuos
ss_residual = total_ss - (ss_A + ss_B + ss_C + ss_D + ss_F + ss_DxF + ss_CxF)

# Cálculo del error de primer orden (error dentro del factor A)
ss_A_error = ss_residual / df_A  # Error dentro del factor A

# Grado de libertad
df_residual = len(df['x']) - (df_A + df_B + df_C + df_D + df_DxF + df_CxF)

# Media de cuadrados（MS）de cada factor
ms_A = ss_A / df_A
ms_B = ss_B / df_B
ms_C = ss_C / df_C
ms_D = ss_D / df_D
ms_F = ss_F / df_F
ms_DxF = ss_DxF / df_DxF
ms_CxF = ss_CxF / df_CxF
ms_residual = ss_residual / df_residual
ms_A_error = ss_A_error / df_A

# Valor de F
f_A = ms_A / ms_residual
f_B = ms_B / ms_residual
f_C = ms_C / ms_residual
f_D = ms_D / ms_residual
f_F = ms_F / ms_residual
f_DxF = ms_DxF / ms_residual
f_CxF = ms_CxF / ms_residual

# Tabla de análisis de varianza (ANOVA)
anova_table = pd.DataFrame({
    'Source': ['A', 'B', 'C', 'D', 'F', 'DxF', 'CxF', 'Residual', 'A Error'],
    'SS': [ss_A, ss_B, ss_C, ss_D, ss_F, ss_DxF, ss_CxF, ss_residual, ss_A_error],
    'df': [df_A, df_B, df_C, df_D, df_F, df_DxF, df_CxF, df_residual, df_A],
    'MS': [ms_A, ms_B, ms_C, ms_D, ms_F, ms_DxF, ms_CxF, ms_residual, ms_A_error],
    'F': [f_A, f_B, f_C, f_D, f_F, f_DxF, f_CxF, np.nan, np.nan],
    'p-value': [p_value_A, p_value_B, p_value_C, p_value_D, p_value_F, p_value_DxF, p_value_CxF, np.nan, np.nan]
})

#
print(anova_table)


In [ ]:
# Conversión a un DataFrame
data = df.copy()

# Conversión de los factores A y B en variables cotegóricas
data['A'] = data['A'].astype('category')
data['B'] = data['B'].astype('category')
data['C'] = data['C'].astype('category')
data['D'] = data['D'].astype('category')
data['F'] = data['F'].astype('category')

# Definición del modelo de análisis de varianza
model = ols('x ~ A +B + DxF + C + D  + CxF + F', data=data).fit()

# Tabla de análisis de varianza (ANOVA)
anova_table = sm.stats.anova_lm(model, typ=2)
anova_table

## 12.8

因子A（２水準）を一次因子、因子B（３水準）を２次因子とした分割実験を反復R（２回）実施して、表12.9のデータを得た。

1. 因子Aの分散比
2. 一次誤差（RxA）の自由度、分散比
3. 因子Bの分散比

を求めよ。

Se realizó un experimento factorial con el factor A (2 niveles) como factor principal y el factor B (3 niveles) como factor secundario, con repetición R (2 veces), obteniendo los datos de la tabla 12.9.

Calcule lo siguiente:
1. Razón de varianza del factor A
2. Grados de libertad del erro primario (RxA), razón de varianza.
3. Razón de varianza del factor B

In [ ]:
# Obtener los datos (como lista)
st = ss.worksheet("Sheet10")
list_of_lists = st.get_all_values()

df = pd.DataFrame.from_records(list_of_lists)
df.columns = df.iloc[0]
df = df.drop(index=0)
df = df.apply(pd.to_numeric)

print("Tabla 12.9 Resultados experimentales")
df

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy.stats import f

# Creación de datos
data_dict = df.to_dict(orient="list")

# Conversión a un DataFrame
data = pd.DataFrame(data_dict)

# Conversión de los factores A y B en variables cotegóricas
data['A'] = data['A'].astype('category')
data['B'] = data['B'].astype('category')
data['R'] = data['R'].astype('category')

# Definición del modelo de análisis de varianza
model = ols('x ~ R + A + R:A + B + A:B', data=data).fit()

# Tabla de análisis de varianza (ANOVA)
anova_table = sm.stats.anova_lm(model, typ=2)

# Calcular la media de los cuadrados
anova_table['mean_sq'] = anova_table['sum_sq'] / anova_table['df']

# Recalcular F0 y el valor p del factor de primer orden
adusts = ['R','A' ]
for adust in adusts:
  anova_table.loc[adust,'F'] = anova_table.loc[adust,'mean_sq']/anova_table.loc['R:A','mean_sq']
  anova_table.loc[adust,'PR(>F)'] = f.sf(anova_table.loc[adust,'F'], anova_table.loc[adust,'df'], anova_table.loc['R:A','df'])

# Visualización de resultados
print("Tabla de análisis de varianza (AVOVA):")
print(anova_table)

# Razón de varianza del factor A
print(f"\n1. Razón de varianza del factor A: {anova_table.loc['A','F']:.2f}")

# Grados de libertad del error primario (R:A)
print(f"2. Grados de libertad del error primario (R:A): {anova_table.loc['R:A','df']}、Razón de varianza: {anova_table.loc['R:A','F']:.2f}")
# Razón de varianza del factor B
print(f"3. Razón de varianza del factor B: {anova_table.loc['B','F']:.2f}")


3. 反復Rは有意ではなく分散比も小さいので、一次誤差（RｘA）にプールし、交互作用AxBは有意ではなく、分散比も小さいので２次誤差（Residual）にプールして分散分析表を作成する。


3. Como la repetición R no es significativa y la razón de varianza es pequeña, se agrupa con el error primario (RxA). Además, dado que la interacción AxB no es significativa y la razón de varianza es pequeña, se agrupa con el error secundario (Residual) para elaborar la tabla de análisis de varianza.

In [ ]:
def pool(anova_table, pool_factors, pool_):
  # Calcular el error agrupado (pooling)
  pooled_error_ss = anova_table.loc[pool_factors, 'sum_sq'].sum()  # Calucar el error agrupado (pooling)
  pooled_error_df = anova_table.loc[pool_factors, 'df'].sum()      # Calucar el grados de libertad agrupado (pooling)

  # Tabla de análisis de varianza de los factores
  anova_table = anova_table.drop(index=pool_factors)

  # Actualizar el término de error
  anova_table.loc[pool_] = [
      anova_table.loc[pool_, 'sum_sq'] + pooled_error_ss,  # Nueva suma de cuadrados del error
      anova_table.loc[pool_, 'df'] + pooled_error_df,      # Nuevos grados de libertad
      None,  # Mean Square
      None   # F-statistic
  ]

  # Calcular la media de los cuadrados
  anova_table['mean_sq'] = anova_table['sum_sq'] / anova_table['df']

  # Calcular el valor F
  anova_table['F'] = anova_table['mean_sq'] / anova_table.loc['Residual', 'mean_sq']

  # Calcular el valor p
  anova_table['PR(>F)'] = f.sf(anova_table['F'], anova_table['df'], anova_table.loc['Residual', 'df'])

  return anova_table

In [ ]:


# Obtener la tabla de análisis de varianza
anova_table = sm.stats.anova_lm(model, typ=2)
anova_table = pool(anova_table,['R'], 'R:A')

anova_table = anova_table.drop(columns=anova_table.columns[-1])
anova_table = pool(anova_table, ['A:B'], 'Residual')

# Recalcular F0 y el valor p del factor de primer orden
adusts = ['A' ]
for adust in adusts:
  anova_table.loc[adust,'F'] = anova_table.loc[adust,'mean_sq']/anova_table.loc['R:A','mean_sq']
  anova_table.loc[adust,'PR(>F)'] = f.sf(anova_table.loc[adust,'F'], anova_table.loc[adust,'df'], anova_table.loc['R:A','df'])

# Mostrar los resultados
print("Tabla 12.11 Tabla de análisis de varianza (después de pooling):")
print(anova_table)

4. 因子A、Bの最適水準組み合わせにおける、母平均と９５％信頼区間をもとめる。


4. Calcular la media poblacional y el intervalo de confianza del 95% para la combinación de niveles de los factores A y B.

In [ ]:
import scipy.stats as stats

# Varianza del error
mse_error = anova_table.loc['Residual', 'mean_sq']

# Valor medio para cada combinación de niveles de factores
group_means = df.groupby(['A', 'B'])['x'].mean()

# Cálcuro de Intervalo de confianza de 95%
n_r = df['R'].nunique()  # Número de repeticiones
df_error = anova_table.loc['Residual', 'df']
t_critical = stats.t.ppf(0.975, df_error)  # Intervalo de confianza de 95%
se_mean = np.sqrt(mse_error / n_r)
ci_lower = group_means - t_critical * se_mean
ci_upper = group_means + t_critical * se_mean

# Combinación de factores en la que el valor
max_idx = group_means.idxmax()
max_mean = group_means[max_idx]
max_ci_lower = ci_lower[max_idx]
max_ci_upper = ci_upper[max_idx]

#
print(f"Factor A={max_idx[0]}, Factor B={max_idx[1]}")
print(f"Media poblacional: {max_mean:.2f}")
print(f"Intervalo de confianza de 95%: ({max_ci_lower:.2f}, {max_ci_upper:.2f})")


## 12.9

わが社は、押し出し成型用のプラスチック原料である製品Qを製造している。Qの重要な品質特性は、成分Rの含有量ｘであるが、最近、納入先より、この含有量のバラつきが大きいのではないかとの苦情が寄せられた。

含有量ｘのバラつき発生状況の調査を行うために次のような実験を行った。

調査は、原料ロットをランダムに３ロット（Ｌ１，Ｌ２，Ｌ３）採取し、各ロットから製造された袋の中から２袋をランダムに選択し、各袋の製品成分Ｒの含有量ｘを２回測定して、表１２．２のデータを得た。



Nuestra empresa fabrica el producto Q, una materia príma plástica para moldeo por extrusión. Una característica de calidad importante de Q es el contenido de R (x). Sin embargo, recientemente hemos recibido quejas de los clientes sobre una posible gran variabilidad en este contenido.

Par investigar la variabilidad en el contenido de x, se realizó el siguiente experimento:

Se seleccionaron aleatriamente tres lotes de materia prima (L1, L2, L3). De cada lote, se eligieron al azar dos sacos de producto, y en cada saco, se midió dos veces el contenido de R, obteniéndose los datos en la Tabla 12.2.



In [ ]:
# Obtener los datos (como lista)
st = ss.worksheet("Sheet11")
list_of_lists = st.get_all_values()

df = pd.DataFrame.from_records(list_of_lists)
df.columns = df.iloc[0]
df = df.drop(index=0)
df = df.apply(pd.to_numeric)

print("Tabla 12.12 Resultados experimentales")
print(df)

μを一般平均、ロット（要因L）によるばらつきを $ \alpha $、袋（要因S）の違いによるばらつきを $ \beta $、測定（要因M）のバラつきを $ \epsilon $ として、データの構造式を作成

1. 分散分析表
2. 要因Sの平方和と自由度、
3. 要因Lに対する分散比、要因Sに対する分散比
4. 要因L、Sの分散成分の推定

Definiendo $ \mu $ como la media general, la variabilidad debida al lote (factor L) como $ \alpha $ la variabilidad debida a las diferencias entre sacos (factor S) como $ \beta $, y la variabilidad en la medición (factor M) como $ \epsilon $, se debe formular la estructura de los datos.

Además se debe calcular lo siguiente:
1. La tabla de análisis de varianza.
2. La suma de cuadrados y los grados de libertad del factor S.
3. La razón de varianza con respecto al factor L y la razó de varianza con respecto al factor S.
4. La estimación de los componentes de varianza de los factores L y S.



* $ x_{ijk} = \mu + \alpha_i + \beta_{ij} + \epsilon_{ijk} $

In [ ]:
# Creación de datos
data_dict = df.to_dict(orient="list")

# Conversión a un DataFrame
data = pd.DataFrame(data_dict)

# Conversión de los factores A y B en variables cotegóricas
data['L'] = data['L'].astype('category')
data['S'] = data['S'].astype('category')
data['M'] = data['M'].astype('category')

# Definición del modelo de análisis de varianza
model = ols('x ~ L + L:S ', data=data).fit()

# Tabla de análisis de varianza (ANOVA)
anova_table = sm.stats.anova_lm(model, typ=2)

anova_table['mean_sq'] = anova_table['sum_sq'] / anova_table['df']

# Recalcular F0 y el valor p del factor de primer orden
adusts = ['L' ]
for adust in adusts:
  anova_table.loc[adust,'F'] = anova_table.loc[adust,'mean_sq']/anova_table.loc['L:S','mean_sq']

print("Tabla 12.18 Tabla de análisis de varianza (AVOVA):")
print(anova_table)

# La suma de cuadrados y los grados de libertad del factor S.
SS = anova_table.loc['L:S', 'sum_sq']
DF = anova_table.loc['L:S', 'df']
print(f"\nSuma de cuadrados del factor S: {SS:.0f}  Grados de libertad: {DF}")

# La razón de varianza con respecto al factor L y la razó de varianza con respecto al factor S.
VL_S = anova_table.loc['L', 'F']
VM_S = anova_table.loc['L:S', 'F']
print(f"Razón de varianza con respecto al factor L: {VL_S:.2f}")
print(f"Razón de varianza con respecto al factor S: {VM_S:.2f}")

La estimación de los componentes de varianza de los factores L y S.

$ E(V_L) = \sigma_M^2 + n \sigma_S^2 + mn \sigma_L^2  $

$ E(V_S) = \sigma_M^2 + \sigma_S^2 $

$ E(V_M) = \sigma_M^2 $

In [ ]:
m = 2
n = 2

VS = anova_table.loc['L:S', 'mean_sq']
VL = anova_table.loc['L', 'mean_sq']
VM = anova_table.loc['Residual', 'mean_sq']

print(f"Estimación del componete de varianza L ：{(VL-VS)/(m*n):.1f}")
print(f"Estimación del componete de varianza S：{(VS-VM)/(n):.1f}")
print(f"Estimación del componete de varianza M：{VM:.1f}")